In [2]:
# ==============================
# 전체 코드: 합성 데이터 생성 + 기사 매칭 + Folium 지도 시각화
# Short Description / 품목 Description 표시
# ==============================

import pandas as pd
import numpy as np
import folium
from folium.plugins import MarkerCluster
from geopy.distance import geodesic

# ==============================
# 1. 파일 경로
# ==============================
file_path = r"C:\Users\enjoy\Study\project\teamproject\FAF5.5.1_HiLoForecasts.csv"
metadata_path = r"C:\Users\enjoy\Study\project\teamproject\FAF5_metadata.xlsx"

# ==============================
# 2. FAF5_metadata.xlsx 읽기
# ==============================
# State 시트: Numeric Label → 주 이름
df_state = pd.read_excel(metadata_path, sheet_name="State")
state_map = df_state.set_index("Numeric Label")["Description"].to_dict()

# FAF Zone 시트: Numeric Label → Short Description
df_zone = pd.read_excel(metadata_path, sheet_name="FAF Zone (Domestic)")
zone_map = df_zone.set_index("Numeric Label")["Short Description"].to_dict()

# Commodity 시트: Numeric Label → Description
df_commodity = pd.read_excel(metadata_path, sheet_name="Commodity (SCTG2)")
commodity_map = df_commodity.set_index("Numeric Label")["Description"].to_dict()

# ==============================
# 3. CSV 읽기 및 필터링
# ==============================
use_cols = [
    "dms_orig", "dms_dest", "dms_mode", "sctg2", "trade_type",
    "tons_2020", "tons_2021", "tons_2022", "tons_2023", 
    "value_2020", "value_2021", "value_2022", "value_2023"
]

chunksize = 1_000_000
filtered_chunks = []

for chunk in pd.read_csv(file_path, usecols=use_cols, chunksize=chunksize):
    chunk_filtered = chunk[(chunk["dms_mode"] == 1) & (chunk["trade_type"] == 1)].copy()
    
    # 연도별 톤당 운임료
    for year in range(2020, 2024):
        chunk_filtered[f"avg_price_per_ton_{year}"] = chunk_filtered[f"value_{year}"] / chunk_filtered[f"tons_{year}"]
    
    # 총 톤수, 총 운임료, 평균 톤당 운임료
    chunk_filtered["tons_total"] = chunk_filtered[["tons_2020","tons_2021","tons_2022","tons_2023"]].sum(axis=1)
    chunk_filtered["value_total"] = chunk_filtered[["value_2020","value_2021","value_2022","value_2023"]].sum(axis=1)
    chunk_filtered["avg_price_per_ton_mean"] = chunk_filtered[[
        "avg_price_per_ton_2020","avg_price_per_ton_2021","avg_price_per_ton_2022","avg_price_per_ton_2023"
    ]].mean(axis=1)
    
    # 성장률
    chunk_filtered["tons_growth"] = (chunk_filtered["tons_2023"] - chunk_filtered["tons_2020"]) / chunk_filtered["tons_2020"]
    chunk_filtered["value_growth"] = (chunk_filtered["value_2023"] - chunk_filtered["value_2020"]) / chunk_filtered["value_2020"]
    
    filtered_chunks.append(chunk_filtered)

# ==============================
# 4. 전체 데이터 합치기
# ==============================
df = pd.concat(filtered_chunks, ignore_index=True)

# Numeric Label → 실제 이름 매핑
df["dms_orig_name"] = df["dms_orig"].map(zone_map)
df["dms_dest_name"] = df["dms_dest"].map(zone_map)
df["commodity_name"] = df["sctg2"].map(commodity_map)

# ==============================
# 5. 기사 매칭용 샘플 생성
# ==============================
np.random.seed(42)
sample_size = 1000
df_sample = df.sample(sample_size, random_state=42).copy()

driver_location = (40.7128, -74.0060)  # 뉴욕

vehicle_types = ["소형", "중형", "대형"]
vehicle_load_ranges = {"소형": (3,5), "중형": (5,10), "대형": (10,15)}

df_sample["vehicle_type"] = np.random.choice(vehicle_types, size=len(df_sample))
df_sample["tons_total"] = df_sample["vehicle_type"].apply(lambda x: np.round(np.random.uniform(*vehicle_load_ranges[x]),1))
n_drivers = 1000
df_sample["driver_id"] = np.random.randint(1, n_drivers+1, size=len(df_sample))

# ==============================
# 6. 운송비용 및 예상 순수익 계산
# ==============================
fuel_price_per_gallon = 4.0
vehicle_mpg = {"소형": 20, "중형": 12, "대형": 6}
km_per_mile = 1.60934

def calc_transport_cost(row):
    distance_mile = row["driver_distance_km"] / km_per_mile
    gallons = distance_mile / vehicle_mpg[row["vehicle_type"]]
    return gallons * fuel_price_per_gallon

df_sample["driver_distance_km"] = df_sample.apply(
    lambda row: geodesic(driver_location, (np.random.uniform(30,50), np.random.uniform(-120,-70))).km,
    axis=1
)

avg_price_per_ton_usd = {"소형": 100, "중형": 150, "대형": 200}
df_sample["expected_profit"] = df_sample.apply(lambda row: row["tons_total"] * avg_price_per_ton_usd[row["vehicle_type"]], axis=1)
df_sample["transport_cost"] = df_sample.apply(calc_transport_cost, axis=1)
df_sample["actual_profit"] = df_sample["expected_profit"] - df_sample["transport_cost"]

# ==============================
# 7. 순수익 > 0 필터링
# ==============================
df_matches = df_sample[df_sample["actual_profit"] > 0].copy()
df_matches_top10 = df_matches.sort_values("actual_profit", ascending=False).head(10)

print("기사 매칭 상위 10건 (순수익 기준)")
print(df_matches_top10[[
    "dms_orig_name","dms_dest_name","commodity_name","vehicle_type",
    "tons_total","expected_profit","driver_distance_km","transport_cost","actual_profit"
]])

# ==============================
# 8. Folium 지도 시각화
# ==============================
m = folium.Map(location=driver_location, zoom_start=5)
folium.Marker(location=driver_location, popup="기사 위치 (New York)", icon=folium.Icon(color="blue", icon="truck", prefix="fa")).add_to(m)

marker_cluster = MarkerCluster().add_to(m)
for idx, row in df_matches.iterrows():
    folium.Marker(
        location=[np.random.uniform(30,50), np.random.uniform(-120,-70)],
        popup=(
            f"출발지: {row['dms_orig_name']} → 도착지: {row['dms_dest_name']}<br>"
            f"품목: {row['commodity_name']}<br>"
            f"차량: {row['vehicle_type']}<br>"
            f"톤수: {row['tons_total']} 톤<br>"
            f"예상 운임: {row['expected_profit']:.2f} USD<br>"
            f"거리: {row['driver_distance_km']:.1f} km<br>"
            f"운송 비용: {row['transport_cost']:.2f} USD<br>"
            f"<b>순수익: {row['actual_profit']:.2f} USD</b>"
        ),
        icon=folium.Icon(color="green", icon="usd", prefix="fa")
    ).add_to(marker_cluster)

m

기사 매칭 상위 10건 (순수익 기준)
                                   dms_orig_name  \
321                  Kansas City MO-KS (KS Part)   
102057                             Greenville SC   
141429  Greensboro--Winston-Salem--High Point NC   
132117  Greensboro--Winston-Salem--High Point NC   
224235                   El Paso TX-NM (TX Part)   
223422                              Nashville TN   
130278      Minneapolis-St. Paul MN-WI (MN Part)   
92780                               Knoxville TN   
264403                                Rest of WI   
31956                                 Rest of OR   

                           dms_dest_name        commodity_name vehicle_type  \
321                           Rest of NE     Live animals/fish           대형   
102057                       Columbus OH           Wood prods.           대형   
141429                         Laredo TX  Nonmetal min. prods.           대형   
132117                        Wichita KS      Textiles/leather           대형   
224235    